In [ ]:
import paramiko
from getpass import getpass
import os

# Password prompt
thePass = getpass(prompt="Enter SSH password: ")

# Host Information
host = "192.168.6.71"
port = 2222
username = "michael.portegello"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username ,password)
except paramiko.AuthenticationException:
    print("Authentication failed.")

In [ ]:
### Task 2 ###

# Upload script
sftp = ssh.open_sftp()
script = "kraken"
sftp.put(script, "/home/michael.portegello/kraken")
sftp.close()

# Execute script
kraken_script = "sudo /home/michael/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin > /home/michael.portegello/output.txt"
stdin, stdout, stderr = ssh.exec_command(kraken_script)
stdin.write(thePass + "/n")

# Get results
sftp = ssh.open_sftp()
sftp.get("/home/michael.portegello/output.txt", "output.txt")

# Exit
sftp.close()
ssh.close()

In [ ]:
### Task 3 ###

# Replace with pid digits
pid = "xxxx"

# Execute lsof commands
lsof_command = ("sudo lsof -p " + pid + " > pid_output.txt")
stdin, stdout, stderr = ssh.exec_command(lsof_command)
stdin.write(thePass + "/n")

# Get results
sftp = ssh.open_sftp()
sftp.get("/home/michael.portegello/pid_output.txt", "pid_output.txt")

# Exit
sftp.close()
ssh.close()

In [ ]:
### Task 4 ###

# Specify files to download
pid_files = ["","","",""]

# Download files
sftp = ssh.open_sftp()
for eachFile in pid_files:
    # Create a list from file name
    x = eachFile.split("/")
    # -1 will get the last element
    filename = x[-1]
    # Local path to store files
    local_path = "suspicious_files/" + filename
    sftp.get(eachFile, local_path)

# Exit
sftp.close()
ssh.close()

In [ ]:
### Task 5 ###

# Check running processes
running_processes = "sudo ps -a"
sftp = ssh.open_sftp()
stdin, stdout, stderr = ssh.exec_command(running_processes)
stdin.write(thePass + "/n")
lines = stdout.readlines()

# Converst list to string
output = '@@@'.join(lines)
# Header
sepHeader = '' + '### BEGIN ' + running_processes + '###\n\n'
# Footer
sepFooter = '' + '### END ' + running_processes + '###\n\n'
# Concatenate
cmd_output = sepHeader + output + sepFooter

# Save output to file
with open('running_processes.txt', 'w') as f:
    f.write(cmd_output)

# Copy /etc/passwd file command
copy_passwd = "sudo cp /etc/passwd /home/michael.portegello/copy_passwd.txt"
# Copy /var/log/auth.log command
copy_authlog = "sudo cp /var/log/auth.log /home/michael.portegello/copy_authlog.txt"

# Execute commands
stdin, stdout, stderr = ssh.exec_command(copy_passwd, copy_authlog)
stdin.write(thePass + "/n")

# Get output files
sftp = ssh.open_sftp()
sftp.get("/home/michael.portegello/copy_passwd.txt", "copy_passwd.txt")
sftp.get("/home/michael.portegello/copy_authlog.txt", "copy_authlog.txt")

# Exit
sftp.close()
ssh.close()